# Spectral Filtering: 2D Butterworth

For images, Butterworth filters are defined to be radial symmetric based on distances. That is, letting 
$d_0$ denote the cut-off frequency distance (-3dB) and $d_{uv} = \sqrt{u^2+v^2}$ the Euclidean distance 
from $(0,0)$ to $(u,v)$, we can define
$$H_{LP}(u,v) = \dfrac{1}{\sqrt{1 + \left( d_{uv}/d_0 \right)^{2n}}}.$$

High-pass filter can be obtained from the low-pass filter: $H^2_{HP}(u,v) = 1-H^2_{LP}(u,v)$. Band-pass filter can be obtained from cascaded low and high-pass filters: $H_{BP}(u,v) = H_{LP}(u,v) H_{HP}(u,v)$.
Band-stop filter can be obtained from band-pass filter: $H^2_{BS}(u,v) = 1-H^2_{BP}(u,v)$.

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from skimage import io, exposure
from skimage.filters import gaussian, window

from scipy.fft import fft2, ifft2, fftshift

from skimage.util import random_noise
from skimage.util import img_as_float32 as img_as_float

In [ ]:
def show_images(I, titles=None):
    fig, ax = plt.subplots(1, len(I), figsize=(12,5))        
 
    for i in np.arange(0,len(I)):
        ax[i].imshow(I[i], cmap='gray')
        ax[i].set_axis_off()
        if titles != None:
            ax[i].set_title(titles[i])
        
    plt.tight_layout()

In [ ]:
def show_plots(I, titles=None):
    fig, ax = plt.subplots(1, len(I), figsize=(12,1))
 
    for i in np.arange(0,len(I)):
        if titles != None:
            ax[i].set_title(titles[i])
        
        r = I[i].shape[0]//2
        ax[i].plot(I[i][r,:])
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        
    plt.tight_layout()

In [ ]:
def bwLP(d0, n, M, N=None):  
    if N == None:
        N = M
    KM, KN = (M//2, N//2)
    u, v = np.mgrid[-KM:KM,-KN:KN]
    
    duv = np.hypot(u, v)
    H = 1/np.sqrt(1+(duv/d0)**(2*n))
    return fftshift(H)

def bwHP(d0, n, M, N=None):
    if N == None:
        N = M
    LP = bwLP(d0, n, M, N)
    return np.sqrt(1-LP**2)

def bwBP(d1, n1, d2, n2, M, N=None):
    if N == None:
        N = M
    LP = bwLP(d2, n2, M, N)
    HP = bwHP(d1, n1, M, N)
    return LP*HP

def bwBS(d1, n1, d2, n2, M, N=None):
    if N == None:
        N = M
    BP = bwBP(d1, n1, d2, n2, M, N)
    return np.sqrt(1-BP**2)

In [ ]:
N = 256

(d0, n0) = (24, 6)
(d1, n1) = (24, 4)
(d2, n2) = (40, 6)

LP = fftshift(bwLP(d0, n0, N))
HP = fftshift(bwHP(d0, n0, N))
BP = fftshift(bwBP(d1, n1, d2, n2, N))
BS = fftshift(bwBS(d1, n1, d2, n2, N))

show_images([np.real(LP), np.real(HP), np.real(BP), np.real(BS)],
            ['Low-Pass','High-Pass','Band-Pass','Band-Stop'])
show_plots([np.real(LP), np.real(HP), np.real(BP), np.real(BS)])

## Example 1: Image Filtering

In [ ]:
def nextpow2(N):
    n = 1
    while (n<N):
        n *= 2
    return n

In [ ]:
I1 = io.imread("../../images/zebras.jpg", as_gray=True)
I1 = img_as_float(I1)

In [ ]:
M, N = I1.shape

N2 = np.max([nextpow2(N), nextpow2(M)])

F1 = fft2(I1,(N2,N2))
G1 = F1*bwLP(200, 6, N2)

I2 = np.real(ifft2(G1))[:M,:N]

show_images([I1, exposure.adjust_log(np.abs(fftshift(F1)))], ['Image', 'FFT: |F|]'])
show_images([I2, exposure.adjust_log(np.abs(fftshift(G1)))], ['Filtered Image', 'FFT: |FH|'])

## Example 2: Image Filtering

In [ ]:
In = random_noise(np.zeros_like(I1), mode='gaussian', clip=False)

Fn = fft2(In, (N2,N2))
Fn = Fn*bwHP(300, 6, N2)

In = np.real(ifft2(Fn))[:M,:N]
Ic = I1 + 0.75*In

Fc = fft2(Ic, (N2,N2))
G2 = Fc*bwLP(256, 6, N2)

I2 = np.real(ifft2(G2))[:M,:N]

show_images([In, exposure.adjust_log(np.abs(fftshift(Fn)))], ['Noise','FFT: |Noise|'])
show_images([Ic, exposure.adjust_log(np.abs(fftshift(Fc)))], ['Corrupted Image','FFT: |F|'])
show_images([I2, exposure.adjust_log(np.abs(fftshift(G2)))], ['Filtered Image','FFT: |FH|'])

## Example 3: Separating Signals

In [ ]:
def sinewaves(M, N, ku, kv):
    T = 2.0*np.pi
    
    KM, KN = (M//2, N//2)
    u, v = np.mgrid[-KM:KM,-KN:KN]
    
    I = np.sin(T*(u*ku/M+v*kv/N))
    
    return I

In [ ]:
#print(M,N,N2)

fu, fv = (100, 200)
duv = np.hypot(fu, fv)

In = 0.25*sinewaves(M, N, (fu*M)/N2, (fv*N)/N2)
Fn = fft2(In, (N2,N2))

Ic = I1 + 0.75*In*window('hamming', In.shape)

Fc = fft2(Ic, (N2,N2))
G2 = Fc*bwBS(duv-50, 12, duv+50, 12, N2)

I2 = np.real(ifft2(G2))[:M,:N]

#show_images([In, np.abs(fftshift(Fn))], ['Noise','FFT: |Noise|'])
show_images([In, exposure.adjust_log(np.abs(fftshift(Fn)))], ['Noise','FFT: |Noise|'])
show_images([Ic, exposure.adjust_log(np.abs(fftshift(Fc)))], ['Corrupted Image','FFT: |F|'])
show_images([I2, exposure.adjust_log(np.abs(fftshift(G2)))], ['Filtered Image','FFT: |FH|'])